# Mask R-CNN - Stats comparison on Inria Aerial Dataset

<i> Sébastien Ohleyer </i>

Comparing FCN, MLP and Mask R-CNN.

Python 3

In [ ]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from scipy.misc import imsave
import imageio
import csv
import pandas as pd
from compute_stats import compute_iou,compute_accuracy
import plotly.plotly as py
import plotly.graph_objs as go
import plotly
plotly.offline.init_notebook_mode(connected=True)

%matplotlib inline 

# Root directory of the project
ROOT_DIR = os.getcwd()

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Directory of Aerial dataset
AERIAL_DIR = "/Users/sebastienohleyer/Documents/ENS MVA/Object recognition/AerialImageDataset/"

In [ ]:
GT_DIR = os.path.join(AERIAL_DIR, "train/gt")
FCN_OUTPUT_DIR = os.path.join(AERIAL_DIR, "train/fcn")
MLP_OUTPUT_DIR = os.path.join(AERIAL_DIR, "train/mlp")
MASKRCNN_OUTPUT_DIR = os.path.join(AERIAL_DIR, "train/maskrcnn_aerial20180115T1637")
MASKRCNN_OUTPUT_DIR2 = os.path.join(AERIAL_DIR, "train/maskrcnn_aerial20180116T1815")

TOWN_LIST = ["austin", "chicago", "kitsap", "tyrol-w", "vienna"]

In [ ]:
results_dict = {}
prediction_methods = ['fcn', 'mlp', 'maskrcnn', 'maskrcnn2', 'maskrcnn3']
for prediction_method in prediction_methods:
    results_dict[prediction_method] = {}
results_dict['fcn']['path'] = os.path.join(AERIAL_DIR, "train/fcn")
results_dict['mlp']['path'] = os.path.join(AERIAL_DIR, "train/mlp")
results_dict['maskrcnn']['path'] = os.path.join(AERIAL_DIR, "train/maskrcnn_aerial20180115T1637")
results_dict['maskrcnn2']['path'] = os.path.join(AERIAL_DIR, "train/maskrcnn_aerial20180116T1815")
results_dict['maskrcnn3']['path'] = os.path.join(AERIAL_DIR, "train/maskrcnn_last")


results_dict['fcn']['color'] = 'rgb(44, 160, 101)'
results_dict['mlp']['color'] = 'rgb(93, 164, 214)'
results_dict['maskrcnn']['color'] = 'rgb(255, 65, 54)'
results_dict['maskrcnn2']['color'] = 'rgb(255, 65, 54)'
results_dict['maskrcnn3']['color'] = 'rgb(255, 65, 54)'

# 1. Image by image

### Init csv file

In [ ]:
writing = False

if writing:
    header = ["method"]
    for town in TOWN_LIST:
        for i in range(1,37):
            image_name = town+str(i)
            header.append(image_name)

    with open('results.csv', 'w') as csvfile:
        spamwriter = csv.writer(csvfile, delimiter=',')
        spamwriter.writerow(header)

### Write csv

In [ ]:
for prediction_method in prediction_methods:
    results_dict[prediction_method][prediction_method+'_acc'] = [prediction_method+'_acc']
    results_dict[prediction_method][prediction_method+'_iou'] = [prediction_method+'_iou']


for town in TOWN_LIST:
    for i in range(1,37):
        image_name = town+str(i)+".tif"
        print(image_name)
        gt = imageio.imread(os.path.join(GT_DIR, image_name) )/255
        
        for prediction_method in prediction_methods:
            if prediction_method=='fcn' or prediction_method=='mlp':
                results_dict[prediction_method]['pred_'+prediction_method] = imageio.imread(os.path.join(results_dict[prediction_method]['path'], prediction_method+'_'+ image_name) )/255
            else:
                results_dict[prediction_method]['pred_'+prediction_method] = imageio.imread(os.path.join(results_dict[prediction_method]['path'], image_name) )/255
        
            results_dict[prediction_method][prediction_method+'_acc'].append(compute_accuracy(gt.astype("uint8"), results_dict[prediction_method]['pred_'+prediction_method].astype("uint8")))
            results_dict[prediction_method][prediction_method+'_iou'].append(compute_iou(gt.astype("uint8"), results_dict[prediction_method]['pred_'+prediction_method].astype("uint8")))

In [ ]:
if writing: 
    with open('results.csv', 'a') as csvfile:
        spamwriter = csv.writer(csvfile, delimiter=',')
        for prediction_method in prediction_methods: 
            spamwriter.writerow(results_dict[prediction_method][prediction_method+'_iou'])
            spamwriter.writerow(results_dict[prediction_method][prediction_method+'_acc'])

### Open csv

In [ ]:
result_data = pd.read_csv("results.csv", index_col=0)
result_data

In [ ]:
np.argmax([result_data[col]["maskrcnn_iou"] for col in result_data.columns])

In [ ]:
np.max([result_data[col]["maskrcnn_iou"] for col in result_data.columns])

In [ ]:
x = []
for town in TOWN_LIST:
    for i in range(1,37):
        x.append(town)

data =[]

for prediction_method in ['fcn', 'mlp', 'maskrcnn']:
    data.append({
        'y': [result_data[col][prediction_method+"_acc"] for col in result_data.columns],
        'x': x,
        'name': prediction_method+"_acc",
        'marker': {
            'color': results_dict[prediction_method]['color'],'size':2
        }, 'boxmean':True,
        "type": "box", 'line': {'width':1}, 'fillcolor':'rgb(255,255,255)',
        'legendgroup' : prediction_method
    })
    data.append({
        'y': [result_data[col][prediction_method+"_iou"] for col in result_data.columns],
        'x': x,
        'name': prediction_method+"_iou",
        'marker': {
            'color': results_dict[prediction_method]['color'],'size':2
        }, 'boxmean':True,
        "type": "box", 'line': {'width':1}, 'fillcolor': None,
        'legendgroup' : prediction_method 
    })
    

layout = {
    'xaxis': {
        'zeroline': False,
    },'boxmode': 'group',
    'margin':dict(l=40,r=30,b=80,t=100,
    ),

}
fig = go.Figure(data=data, layout=layout)

plotly.offline.iplot(fig, image='webp')

# 2. Over entire towns

#### Init csv file

In [ ]:
writing2 = True


In [ ]:
if writing2 : 
    header =['method']+TOWN_LIST+['overall']

    with open('results_overall.csv', 'w') as csvfile:
        spamwriter = csv.writer(csvfile, delimiter=',')
        spamwriter.writerow(header)

In [ ]:
results_overall_dict = {}
predictions_methods = ['fcn', 'mlp', 'maskrcnn', 'maskrcnn2','maskrcnn3']
for prediction_method in predictions_methods:
    results_overall_dict[prediction_method] = {}
results_overall_dict['fcn']['path'] = os.path.join(AERIAL_DIR, "train/fcn")
results_overall_dict['mlp']['path'] = os.path.join(AERIAL_DIR, "train/mlp")
results_overall_dict['maskrcnn']['path'] = os.path.join(AERIAL_DIR, "train/maskrcnn_aerial20180115T1637")
results_overall_dict['maskrcnn2']['path'] = os.path.join(AERIAL_DIR, "train/maskrcnn_aerial20180116T1815")
results_overall_dict['maskrcnn3']['path'] = os.path.join(AERIAL_DIR, "train/maskrcnn_last")

In [ ]:
from numpy import count_nonzero as nnz


for prediction_method in predictions_methods:
    results_overall_dict[prediction_method][prediction_method+'_acc'] = [prediction_method+'_acc']
    results_overall_dict[prediction_method][prediction_method+'_iou'] = [prediction_method+'_iou']
    
    results_overall_dict[prediction_method][prediction_method+'_inters_acum'] = 0
    results_overall_dict[prediction_method][prediction_method+'_union_acum'] = 0
    results_overall_dict[prediction_method][prediction_method+'_correct_acum'] = 0
    results_overall_dict[prediction_method][prediction_method+'_total_acum'] = 0

for town in TOWN_LIST:
    print(town)
    for prediction_method in predictions_methods:
        results_overall_dict[prediction_method][prediction_method+'_inters_count'] = 0
        results_overall_dict[prediction_method][prediction_method+'_union_count'] = 0
        results_overall_dict[prediction_method][prediction_method+'_correct_count'] = 0
        results_overall_dict[prediction_method][prediction_method+'_total_count'] = 0
    

    for i in range(5,37):
        image_name = town+str(i)+".tif"
        print(image_name)
        gt = imageio.imread(os.path.join(GT_DIR, image_name) )/255
        
        for prediction_method in predictions_methods:
            if prediction_method=='fcn' or prediction_method=='mlp':
                results_overall_dict[prediction_method]['pred_'+prediction_method] = imageio.imread(os.path.join(results_overall_dict[prediction_method]['path'], prediction_method+'_'+ image_name) )/255
            else:
                results_overall_dict[prediction_method]['pred_'+prediction_method] = imageio.imread(os.path.join(results_overall_dict[prediction_method]['path'], image_name) )/255
            
            results_overall_dict[prediction_method][prediction_method+'_inters'] = gt.astype("uint8") & results_overall_dict[prediction_method]['pred_'+prediction_method].astype("uint8")
            results_overall_dict[prediction_method][prediction_method+'_union'] = gt.astype("uint8") | results_overall_dict[prediction_method]['pred_'+prediction_method].astype("uint8")
            results_overall_dict[prediction_method][prediction_method+'_correct'] = gt.astype("uint8") == results_overall_dict[prediction_method]['pred_'+prediction_method].astype("uint8")

            results_overall_dict[prediction_method][prediction_method+'_inters_count'] += nnz(results_overall_dict[prediction_method][prediction_method+'_inters'])
            results_overall_dict[prediction_method][prediction_method+'_union_count'] += nnz(results_overall_dict[prediction_method][prediction_method+'_union'])
            results_overall_dict[prediction_method][prediction_method+'_correct_count'] += nnz(results_overall_dict[prediction_method][prediction_method+'_correct'])
            results_overall_dict[prediction_method][prediction_method+'_total_count'] += gt.size
            
    for prediction_method in predictions_methods:       
        results_overall_dict[prediction_method][prediction_method+'_inters_acum'] += results_overall_dict[prediction_method][prediction_method+'_inters_count']
        results_overall_dict[prediction_method][prediction_method+'_union_acum'] += results_overall_dict[prediction_method][prediction_method+'_union_count']
        results_overall_dict[prediction_method][prediction_method+'_correct_acum'] += results_overall_dict[prediction_method][prediction_method+'_correct_count']
        results_overall_dict[prediction_method][prediction_method+'_total_acum'] += results_overall_dict[prediction_method][prediction_method+'_total_count']

        results_overall_dict[prediction_method][prediction_method+'_iou'].append(results_overall_dict[prediction_method][prediction_method+'_inters_count']/float(results_overall_dict[prediction_method][prediction_method+'_union_count']))
        results_overall_dict[prediction_method][prediction_method+'_acc'].append(results_overall_dict[prediction_method][prediction_method+'_correct_count']/float(results_overall_dict[prediction_method][prediction_method+'_total_count']))

for prediction_method in predictions_methods:
    results_overall_dict[prediction_method][prediction_method+'_iou'].append(results_overall_dict[prediction_method][prediction_method+'_inters_acum']/float(results_overall_dict[prediction_method][prediction_method+'_union_acum']))
    results_overall_dict[prediction_method][prediction_method+'_acc'].append(results_overall_dict[prediction_method][prediction_method+'_correct_acum']/float(results_overall_dict[prediction_method][prediction_method+'_total_acum']))

In [ ]:
if writing2:
    with open('results_overall.csv', 'a') as csvfile:
        spamwriter = csv.writer(csvfile, delimiter=',')
        for prediction_method in predictions_methods: 
            spamwriter.writerow(results_overall_dict[prediction_method][prediction_method+'_iou'])
            spamwriter.writerow(results_overall_dict[prediction_method][prediction_method+'_acc'])